# Introduction: Automated Feature Engineering

In this notebook, we will look at an exciting development in data science: automated feature engineering. A machine learning model can only learn from the data we give it, and making sure that data is relevant to the task is one of the most crucial steps in the machine learning pipeline. The importance of creating useful features from existing data has been highlighted by numerous leaders in the field, including Andrew Ng ("applied machine learning is basically feature engineering") and in the excellent paper "A Few Useful Things to Know about Machine Learning". However, manual feature engineering is a tedious task and is limited by both human imagination - there are only so many features we can think to create - and by time - creating new features is time-intensive. Ideally, there would be an objective method to create an array of diverse new features that we can then use for a machine learning task. 

In this notebook, we will walk through an implementation of using Feature Tools, an open-source Python library for automatically creating features with relational data (where the data is in structured tables). Although there are now many efforts working to enable automated model selection and hyperparameter tuning, there has been a lack of work on the feature engineering aspect of the pipeline. This library seeks to close that gap and the general methodology has been proven effective in both machine learning competitions and business use cases. 


## Dataset

We will use an example dataset consisting of three tables:

* `clients`: information about clients at a credit union
* `loans`: previous loans taken out by the clients
* `payments`: payments made/missed on the previous loans

The general problem of feature engineering is taking disparate data, often distributed across multiple tables, and combining it into a single table that can be used for training a machine learning model. If we were doing this manually, we could use `pandas` functions to group the data by client and then calculate summary statistics. Feature tools can do this exact same process, but will create far more features than we would have considered.

First, let's load in the data and look at how we might create new features by hand to contrast with feature tools.

In [1]:
# Run this if feature tools is not already installed
# !pip install featuretools

In [2]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Read in the data
clients = pd.read_csv('data/clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('data/loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('data/payments.csv', parse_dates = ['payment_date'])

In [4]:
clients.head()

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [5]:
loans.sample(10)

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
131,32726,credit,14522,0,10202,2005-06-23,2007-07-01,3.74
229,32961,home,9237,0,10850,2011-11-25,2014-07-27,0.01
402,41472,cash,5758,0,11711,2003-02-04,2005-09-27,1.88
10,46109,other,1618,0,11661,2006-08-28,2009-04-23,6.49
416,49624,cash,13221,1,11581,2003-05-07,2006-01-06,3.94
339,39384,cash,8646,1,10538,2014-03-10,2016-11-27,1.84
134,32726,other,1710,1,10522,2000-06-12,2002-05-08,0.31
167,35214,home,667,0,11731,2007-02-01,2009-04-06,5.11
409,41472,other,11773,0,10364,2001-05-09,2003-12-30,6.19
341,39384,home,7241,1,11972,2009-03-28,2011-03-09,5.83


In [6]:
payments.sample(10)

,loan_id,payment_amount,payment_date,missed
395,11430,136,2009-11-23,1
3216,11238,1682,2001-06-06,0
3413,10330,1992,2002-04-27,1
159,11030,2314,2001-10-14,1
574,10205,938,2010-04-30,1
204,11043,2505,2003-02-18,1
1723,10239,1435,2014-01-20,0
3307,10826,2538,2005-03-20,0
2553,11100,473,2014-10-07,0
1061,11287,1556,2007-05-26,1


### Manual Feature Engineering Examples

The first features we might make by hand are relatively simple: we can take the month of the `joined` column and the natural log of the `income` column. Later, we will come to see these are known in Feature Tools as transformation feature primitives because they act on all values in a column. 

In [7]:
# Create a month column
clients['join_month'] = clients['joined'].dt.month

# Create a log of income column
clients['log_income'] = np.log(clients['income'])

clients.head()

,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739
3,46180,2001-11-06,43851,562,11,10.688553
4,25707,2006-10-06,211422,621,10,12.261611


To incorporate information about the other tables, we would use the `df.groupby` method, followed by a suitable aggregation function, followed by `df.merge`.  For example, let's calculate the average, minimum, and maximum amount of previous loans for each client. In the terms of feature tools, this would be considered an aggregation feature primitive because we are aggregating a statistic for multiple data points.

In [8]:
# Groupby client id and calculate mean, max, min previous loan size
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']
stats.head()

,mean_loan_amount,max_loan_amount,min_loan_amount
client_id,,,
25707,7963.950000,13913,1212
26326,7270.062500,13464,1164
26695,7824.722222,14865,2389
26945,7125.933333,14593,653
29841,9813.000000,14837,2778


In [9]:
# Merge with the clients dataframe
clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head(10)

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount
0,46109,2002-04-16,172677,527,4,12.059178,8951.600000,14049,559
1,49545,2007-11-14,104564,770,11,11.557555,10289.300000,14971,3851
2,41480,2013-03-11,122607,585,3,11.716739,7894.850000,14399,811
3,46180,2001-11-06,43851,562,11,10.688553,7700.850000,14081,1607
4,25707,2006-10-06,211422,621,10,12.261611,7963.950000,13913,1212
5,39505,2011-10-14,153873,610,10,11.943883,7424.050000,14575,904
6,32726,2006-05-01,235705,730,5,12.370336,6633.263158,14802,851
7,35089,2010-03-01,131176,771,3,11.784295,6939.200000,13194,773
8,35214,2003-08-08,95849,696,8,11.470529,7173.555556,14767,667
9,48177,2008-06-09,190632,769,6,12.158100,7424.368421,14740,659


If we then wanted to include information about the `payments` we would have to group that dataframe by the `loan_id`, merge it with the `loans`, group the resulting dataframe by the `client_id`, and then merge it into the `clients` dataframe. This would allow us to include information about previous payments for each client. Clearly, this process can grow quite tedious with multiple tables and I certainly don't want to have to do this process by hand! Luckily, feature tools can automatically carry out this entire process and will create more features than we would have ever thought of. Although I love `pandas`, there is only so much manual data manipulation I'm willing to stand! 

# Feature Tools

Now that we know what we are trying to avoid, let's figure out how to automate this process. Feature tools takes the human limits of time and imagination out of the manual feature engineering process (although it is meant to be used together with a data scientist and not replace them!) Feature tools operates on an idea known as Deep Feature Synthesis. You can read the original paper here, and although it's quite readable, it's not necessary to understand the details to do automated feature engineering. Basically, feature tools uses basic building blocks known as feature primitives (like those above) that when combined together can yield multiple new variables. These variables can then be used for supervised machine learning. 

I threw out some terms there, but don't worry because we'll cover them as we go. Feature Tools builds on simple ideas to create a powerful method, and we will build up our understanding in much the same way. 

The first part of Feature Tools to understand is an `entity`. This is simply a table, or in `pandas`, a `DataFrame`. We corral multiple entities into a single object called an `EntitySet`. This is a data structure composed of many entities and is quite useful for Deep Feature Synthesis. The `EntitySet` will hold all of our entities (think tables) and the relationships between them. 

## EntitySet

Creating a new `EntitySet` is pretty simple: 

In [10]:
es = ft.EntitySet(id = 'clients')

## Entities 

An entity is simply a table, which is represented in Pandas as a `dataframe`. Each entity must have a uniquely identifying column, known as an index. For the clients dataframe, this is the `client_id` because each id only appears once in the data. In the `loans` dataframe, `client_id` is not an index because each id might appear more than once. The index for this dataframe is instead `loan_id`. When we create an `entity` in feature tools, we have to identify which column of the dataframe is the index. 

If the data does not have a unique index we can tell feature tools to make an index for the entity by passing in `make_index = True` and specifying a name for the index. 

If the data also has a uniquely identifying time index, we can pass that in as the `time_index` parameter. Feature tools will automatically infer the variable types (numeric, categorical, datetime) of the columns in our data, but we can also pass in specific datatypes to override this behavior. As an example, even though the `repaid` column in the `loans` dataframe is represented as an integer, we can tell feature tools that this is a categorical feature since it can only take on two discrete values. 

In the code below we create the three entities and add them to the `EntitySet`.  The syntax is relatively straightforward with a few notes: for the `payments` dataframe we need to make an index, and for the `loans` dataframe, we specify that `repaid` is a categorical variable. 

In [11]:
# Create an entity from the client dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')

In [12]:
# Create an entity from the loans dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, 
                              variable_types = {'repaid': ft.variable_types.Categorical},
                              index = 'loan_id', time_index = 'loan_start')

In [13]:
# Create an entity from the payments dataframe
# This does not yet have a unique index
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              variable_types = {'missed': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')

In [14]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

All three entities have been successfully added to the `EntitySet`. We can access any of these using Python dictionary syntax.

In [15]:
es['loans']

Entity: loans
  Variables:
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    loan_start (dtype: datetime_time_index)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
    repaid (dtype: categorical)
    loan_id (dtype: index)
  Shape:
    (Rows: 443, Columns: 8)

Feature tools correctly inferred each of the datatypes when we made this entity. We can also see that we overrode the type for the `repaid` feature, changing if from numeric to categorical. 

## Relationships

After defining the entities (tables) in an `EntitySet`, we now need to tell feature tools how they are related. The most intuitive way to think of relationships is with the parent to child analogy: a parent-to-child relationship is one-to-many because for each parent, there can be multiple children. The `client` dataframe is therefore the parent of the `loans` dataframe because while there is only one row for each client in the `client` dataframe, each client may have several previous loans covering multiple rows in the `loans` dataframe. Likewise, the `loans` dataframe is the parent of the `payments` dataframe because each loan will have multiple payments. 

These relationships are what allow us to group together datapoints (called aggregating) and then create new features. As an example, we can group all of the previous loans associated with one client and find the average loan amount. We will discuss the features themselves more in a little bit, but for now let's define the relationships. To define relationships, we need to specify the parent variable and the child variable. This is the variable that links two entities together. In our example, the `client` and `loans` dataframes are linked together by the `client_id` column. Again, this is a parent to child relationship because for each `client_id` in the parent `client` dataframe, there may be multiple entries of the same `client_id` in the child `loans` dataframe. 

We codify relationships in the language of feature tools by specifying the parent variable and then the child variable. After creating a relationship, we add it to the `EntitySet`. 

In [16]:
# Relationship between clients and previous loans
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_client_previous)

es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id

The relationship has now been stored in the entity set. The second relationship is between the `loans` and `payments`. These two entities are related by the `loan_id` variable.

In [17]:
# Relationship between previous loans and previous payments
r_payments = ft.Relationship(es['loans']['loan_id'],
                                      es['payments']['loan_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_payments)

# Deep Feature Synthesis

In the paper describing the original idea behind feature tools, the authors named the process of creating new features they developed "Deep Feature Synthesis". The deep in the title refers to the idea that features can have different depth depending on the number of feature primitives used. What is a feature primitive? Well, since you asked: 


## Feature Primitives

A feature primitive is an operation acting on a feature. These fall into two categories:

* __Aggregation__: function that groups together child datapoints for each parent and then calculates a statistic such as mean, min, max, or standard deviation. An example is calculating the maximum loan amount for each client. 
* __Transformation__: an operation applied to every observation in a feature. An example would be extracting the day from dates, or the square root of a numeric. In contrast to aggregations, a transformation does not group observations.


Let's take a look at feature primitives in feature tools. We can view the list of primitives:

In [18]:
primitives = ft.list_primitives()
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,description
0,n_most_common,aggregation,Finds the N most common elements in a categori...
1,std,aggregation,Finds the standard deviation of a numeric feat...
2,avg_time_between,aggregation,Computes the average time between consecutive ...
3,max,aggregation,Finds the maximum non-null value of a numeric ...
4,all,aggregation,Test if all values are 'True'.
5,num_true,aggregation,Finds the number of 'True' values in a boolean.
6,mode,aggregation,Finds the most common element in a categorical...
7,percent_true,aggregation,Finds the percent of 'True' values in a boolea...
8,num_unique,aggregation,Returns the number of unique categorical varia...
9,trend,aggregation,Calculates the slope of the linear trend of va...


In [19]:
primitives[primitives['type'] == 'transform'].head(10)

,name,type,description
19,longitude,transform,Returns the second value on the tuple base fea...
20,latitude,transform,Returns the first value of the tuple base feat...
21,percentile,transform,"For each value of the base feature, determines..."
22,month,transform,Transform a Datetime feature into the month.
23,cum_sum,transform,Calculates the sum of previous values of an in...
24,week,transform,Transform a Datetime feature into the week.
25,diff,transform,Compute the difference between the value of a ...
26,haversine,transform,Calculate the approximate haversine distance i...
27,numwords,transform,Returns the words in a given string by countin...
28,weekday,transform,Transform Datetime feature into the boolean of...


If feature tools does not have enough primitives for us, we can also make our own. 

To get an idea of what a feature primitive actually does, let's try out a few on our data. Using primitives is surprisingly easy using the `ft.dfs` function (which stands for deep feature synthesis). In this function, we specify the entityset to use; the `target_entity`, which is the dataframe we want to make the features for; the `agg_primitives` which are the aggregation feature primitives; and the `trans_primitives` which are the transformation primitives to apply. 

In the following example, we are using the `EntitySet` we already created, the target entity is the `clients` dataframe because we want to make new features about each client, and then we specify a few aggregation and transformation primitives. 

In [20]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['years', 'month', 'subtract', 'divide'])

In [21]:
features.head()

,income,credit_score,join_month,log_income,MEAN(loans.loan_amount),MEAN(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),LAST(loans.loan_type),LAST(loans.loan_amount),...,income / income - log_income,credit_score - log_income / join_month,log_income - income / log_income - join_month,income - log_income / LAST(payments.payment_amount),income - credit_score / join_month,MAX(loans.rate) / MEAN(loans.loan_amount),MAX(payments.payment_amount) / LAST(loans.rate),credit_score - log_income / LAST(loans.loan_amount),credit_score - join_month / credit_score - log_income,income - credit_score / LAST(loans.loan_amount)
client_id,,,,,,,,,,,,,,,,,,,,,
25707,211422,621,10,12.261611,7963.950000,3.477000,13913,9.44,home,2203,...,1.000058,60.873839,-93477.481134,877.218831,21080.100000,0.001185,365.405405,0.276322,1.003715,95.688153
26326,227920,633,5,12.336750,7270.062500,2.517500,13464,6.73,credit,5275,...,1.000054,124.132650,-31063.844919,244.536119,45457.400000,0.000926,1833.103448,0.117661,1.011821,43.087583
26695,174532,680,8,12.069863,7824.722222,2.466111,14865,6.51,other,13918,...,1.000069,83.491267,-42881.029074,83.742769,21731.500000,0.000832,3257.777778,0.047990,1.006093,12.491163
26945,214516,806,11,12.276140,7125.933333,2.855333,14593,5.65,cash,9249,...,1.000057,72.156715,-168087.976224,134.316671,19428.181818,0.000793,967.832168,0.085817,1.001608,23.106282
29841,38354,523,8,10.554614,9813.000000,3.445000,14837,6.76,home,7223,...,1.000275,64.055673,-15009.486297,47.929307,4728.875000,0.000689,569.351670,0.070946,1.004985,5.237574


Already we can see how useful feature tools is: it performed the same operations we did manually but also many more in addition. Examining the dataframe brings us to the final piece of the puzzle: deep features.

## Feature Depth

While feature primitives are useful by themselves, the main benefit of using feature tools arises when we stack primitives to get deep features. The depth of a feature is simply the number of primitives required to make a feature. So, a feature that relies on a single aggregation would be a deep feature with a depth of 1. The idea itself is lot simpler than the name "deep feature synthesis" implies. (I think the authors were trying to ride the way of deep neural network hype when they named the method!)

Already in the dataframe we made by specifying the primitives manually we can see the idea of feature depth. For instance, the MEAN(loans.loan_amount) feature has a depth of 1 because it is made by applying a single aggregation primitive. This feature represents the average size of a client's previous loans.


In [22]:
# Show a feature with a depth of 1
features['MEAN(loans.loan_amount)'].head(10)

client_id
25707    7963.950000
26326    7270.062500
26695    7824.722222
26945    7125.933333
29841    9813.000000
32726    6633.263158
32885    9920.400000
32961    7882.235294
35089    6939.200000
35214    7173.555556
Name: MEAN(loans.loan_amount), dtype: float64

As well scroll through the features, we see a number of features with a depth of 2. For example, the LAST(loans.(MEAN(payments.payment_amount))) has depth = 2 because it is made by stacking two feature primitives, first an aggregation and then a transformation. This feature represents the average payment amount for the last (most recent) loan for each client.

In [23]:
# Show a feature with a depth of 2
features['LAST(loans.MEAN(payments.payment_amount))'].head(10)

client_id
25707     293.500000
26326     977.375000
26695    1769.166667
26945    1598.666667
29841    1125.500000
32726     799.500000
32885    1729.000000
32961     282.600000
35089     110.400000
35214    1410.250000
Name: LAST(loans.MEAN(payments.payment_amount)), dtype: float64

We can create features of arbitrary depth by stacking more primitives. However, when I have used feature tools I've never gone beyond a depth of 2! After this point, the features become very convoluted to understand. I'd encourage anyone interested to experiment with increasing the depth (maybe for a real problem) and see if there is value to "going deeper".

## Automated Deep Feature Synthesis

The main benefit of using feature tools comes not in specifying aggregations and transformations by hand, but by letting feature tools automatically generate many new features. We can do this by making the same `ft.dfs` function call, but without passing in any primitives. We can just set the `max_depth` parameter and feature tools will automatically try out all combinations of feature primitives to the ordered depth. 

When running on large datasets, this process can take quite a while, but for our example data, it will be relatively quick. For this call, we only need to specify the `entityset`, the `target_entity` (which will again be `clients`), and the `max_depth`. 

In [24]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2)

In [25]:
features.head()

,income,credit_score,join_month,log_income,SUM(loans.loan_amount),SUM(loans.rate),STD(loans.loan_amount),STD(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),...,NUM_UNIQUE(loans.WEEKDAY(loan_end)),MODE(loans.MODE(payments.missed)),MODE(loans.DAY(loan_start)),MODE(loans.DAY(loan_end)),MODE(loans.YEAR(loan_start)),MODE(loans.YEAR(loan_end)),MODE(loans.MONTH(loan_start)),MODE(loans.MONTH(loan_end)),MODE(loans.WEEKDAY(loan_start)),MODE(loans.WEEKDAY(loan_end))
client_id,,,,,,,,,,,,,,,,,,,,,
25707,211422,621,10,12.261611,159279,69.54,4044.418728,2.421285,13913,9.44,...,6,0,27,1,2010,2007,1,8,3,0
26326,227920,633,5,12.336750,116321,40.28,4254.149422,1.991819,13464,6.73,...,5,0,6,6,2003,2005,4,7,5,2
26695,174532,680,8,12.069863,140845,44.39,4078.228493,1.517660,14865,6.51,...,6,0,3,14,2003,2005,9,4,1,1
26945,214516,806,11,12.276140,106889,42.83,4389.555657,1.564795,14593,5.65,...,6,0,16,1,2002,2004,12,5,0,1
29841,38354,523,8,10.554614,176634,62.01,4090.630609,2.063092,14837,6.76,...,7,1,1,15,2005,2007,3,2,5,1


Deep feature synthesis has created 94 new features out of the existing data! While we could have created all of these manually, I am glad to not have to write all that code by hand. The primary benefit of feature tools is that it creates features without any subjective human biases. Even a human with considerable domain knowledge will be limited by their imagination when making new features (not to mention time). Automated feature engineering is not limited by any factors and provides a good starting point for feature creation. 

While, automatic feature engineering solves one problem, it provides us with another problem: too many features! Although it's impossible to say which features will be important to a given machine learning task, it's likely that not all of the features made by feature tools add value. In fact, having too many features is a significant issue in machine learning because it makes training a model much harder. The irrelevant features can drown out the important features leaving a model unable to learn how to map the features to the target. This problem is known as the "curse of dimensionality" and is addressed through the process of feature reduction, which means removing low-value features from the data. Feature reduction will have to be another topic for another day!

In [26]:
features.shape

(25, 94)

# Conclusions

In this notebook, we saw how to apply automated feature engineering to an example dataset. This is a powerful method which allows us to overcome the human limits of time and imagination to create many new features. Feature tools is built on the idea of deep feature synthesis, which means stacking multiple simple feature primitives - aggregations and transformations - to create new features. Feature engineering allows us to combine information across many tables into a single dataframe that we can then use for machine learning model training. The next step after creating all of these features is figuring out which ones are important. 

Feature tools is currently the only Python option for this process, but with the recent emphasis on automating aspects of the machine learning pipeline, other competitiors will probably enter the sphere. While the exact tools will change, the idea of automatically creating new features out of existing data will grow in importance. Staying up-to-date on methods such as automated feature engineering is crucial in the rapidly changing field of data science. Now go out there and find a problem on which to apply feature tools! 